# VacationPy
---



### Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vammala,61.3333,22.9000,0.25,89,81,2.69,FI,1706978496
1,1,port elizabeth,-33.9180,25.5701,24.50,91,100,3.09,ZA,1706978293
2,2,eydhafushi,5.1033,73.0708,27.98,78,72,5.44,MV,1706978498
3,3,vilyuysk,63.7553,121.6247,-28.89,99,95,2.24,RU,1706978498
4,4,alo,7.4178,6.7371,36.68,12,58,1.85,NG,1706978500


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
city_map = city_data_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    size ="Humidity",
    hover_cols = ['City'],
    geo=True,
    tiles='OSM',
    color = "City",
    scale = 0.5,
    frame_width = 800,
    frame_height = 500
    )

city_map

c:\Users\Albrecht\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
new_city_data_df = city_data_df[
    (city_data_df['Max Temp'] <50) &
    (city_data_df['Humidity'] <70) &
    (city_data_df['Wind Speed'] < 5)
]

new_city_data_df = new_city_data_df.dropna()

new_city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,alo,7.4178,6.7371,36.68,12,58,1.85,NG,1706978500
11,11,yangambi,0.8102,24.4336,29.95,36,68,0.42,CD,1706978507
19,19,alice springs,-23.7000,133.8833,29.75,30,75,3.60,AU,1706978513
28,28,aykhal,66.0000,111.5000,-27.93,64,83,1.67,RU,1706978424
30,30,murzuq,25.9155,13.9184,13.10,27,0,3.00,LY,1706978522
...,...,...,...,...,...,...,...,...,...,...
565,565,san felipe,10.3399,-68.7425,31.92,56,89,0.74,VE,1706978984
573,573,itupiranga,-5.1347,-49.3267,29.18,67,90,1.11,BR,1706978992
579,579,lethem,3.3803,-59.7968,38.52,23,40,2.46,GY,1706978998
581,581,haffouz,35.6324,9.6762,15.57,62,6,3.57,TN,1706979000


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
hotel_df = new_city_data_df[['City','Country','Lat','Lng','Humidity']].copy()

hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,alo,NG,7.4178,6.7371,12,
11,yangambi,CD,0.8102,24.4336,36,
19,alice springs,AU,-23.7000,133.8833,30,
28,aykhal,RU,66.0000,111.5000,64,
30,murzuq,LY,25.9155,13.9184,27,
...,...,...,...,...,...,...
565,san felipe,VE,10.3399,-68.7425,56,
573,itupiranga,BR,-5.1347,-49.3267,67,
579,lethem,GY,3.3803,-59.7968,23,
581,haffouz,TN,35.6324,9.6762,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
radius = 10000
category = "accommodation.hotel"
conditions = "internet_access"
params = {"categories":category,
          "conditions":conditions,
          "apiKey":geoapify_key,
          }

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format

    place_data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = place_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alo - nearest hotel: No hotel found
yangambi - nearest hotel: No hotel found
alice springs - nearest hotel: Crowne Plaza Lasseter's
aykhal - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
mazouna - nearest hotel: No hotel found
dunedin - nearest hotel: Scenic Hotel Dunedin City
whitehorse - nearest hotel: Best Western Gold Rush Inn
mermoz boabab - nearest hotel: Casa Mara Dakar
oltinko'l - nearest hotel: No hotel found
badger - nearest hotel: Candlewood Suites Fairbanks
beledweyne - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Porto Hotel
chui - nearest hotel: Nuevo Hotel Plaza
aiken - nearest hotel: No hotel found
urucui - nearest hotel: No hotel found
maracaibo - nearest hotel: Hotel Caribe
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
tucupita - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
hasaki - nearest hotel: ペンションヘルシーウィング
tome acu

,City,Country,Lat,Lng,Humidity,Hotel Name
4,alo,NG,7.4178,6.7371,12,No hotel found
11,yangambi,CD,0.8102,24.4336,36,No hotel found
19,alice springs,AU,-23.7000,133.8833,30,Crowne Plaza Lasseter's
28,aykhal,RU,66.0000,111.5000,64,No hotel found
30,murzuq,LY,25.9155,13.9184,27,No hotel found
...,...,...,...,...,...,...
565,san felipe,VE,10.3399,-68.7425,56,No hotel found
573,itupiranga,BR,-5.1347,-49.3267,67,No hotel found
579,lethem,GY,3.3803,-59.7968,23,ORI hotel
581,haffouz,TN,35.6324,9.6762,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:


hotel_map = hotel_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    size ="Humidity",
    hover_cols = ['City','Hotel Name','Country'],
    geo=True,
    tiles='OSM',
    color = "City",
    scale = 0.5,
    frame_width = 800,
    frame_height = 500
    )

hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)